In [39]:
import os
from dataclasses import dataclass, field
import datetime

from cli import cli


@dataclass
class SES2000_Depth:
    datetime_start: str = ''

    lat: list = field(default_factory=list)
    lon: list = field(default_factory=list)
    timestamp: list = field(default_factory=list)
    time: list = field(default_factory=list)
    date: list = field(default_factory=list)
    sv: list = field(default_factory=list)
    depth: list = field(default_factory=list)



path = r'C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack'



out_files = cli.search(path, '.out')

Searching *.out files in directory:C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack
Scanned filesets are: 

0 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput-31-10-2024.out
1 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput10302024.out
2 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput11062024.out
3 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput11082024.out
4 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput11122024.out
5 C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\SES_Bathymetry_RAW_FromHypack\SMOutput11172024.out
6 C:\YandexDisk\aa_cloudmbespr

In [40]:
depth_objs = []


for file in out_files:
    with open(file, 'r') as f1:
        
        depth_obj = SES2000_Depth()
        
        f1_content = f1.read().splitlines()
        
        for line in f1_content:
            if line.startswith('$USR'):
                line_content = line[4:].split(',')
                
                lat = float(line_content[0])
                lon = float(line_content[1])
                
                if lon > 0:
                    lon = lon*-1

                    
                sv = 1517.0
                
                depth = float(line_content[-5])
                time = line_content[-2]
                date = line_content[-1]
                
                datetime_obj = datetime.datetime.strptime(time[:8] + '_' + date, '%H:%M:%S_%m/%d/%Y')
                datetime_obj.replace(tzinfo=datetime.timezone.utc)
                timestamp = datetime_obj.timestamp()
                
            elif line.startswith('AI69'):
                line_content = line.split(',')
                
                lat = float(line_content[1][0:2]) + float(line_content[1][2:])/60
                lon = float(line_content[2][0:3]) + float(line_content[2][3:])/60
                
                if lon > 0:
                    lon = lon*-1
                    
                sv = float(line_content[5])
                
                if sv == 0:
                    sv = 1517.0
                
                depth = float(line_content[3])
                time = line_content[-1]
                date = line_content[-2]
                
                datetime_obj = datetime.datetime.strptime(time[:8] + '_' + date, '%H:%M:%S_%m/%d/%Y')
                datetime_obj.replace(tzinfo=datetime.timezone.utc)
                timestamp = datetime_obj.timestamp()
                
            else:
                pass
            
            depth_obj.lat.append(lat)
            depth_obj.lon.append(lon)
            depth_obj.time.append(time)
            depth_obj.date.append(date)
            depth_obj.sv.append(sv)
            depth_obj.depth.append(depth)
            depth_obj.timestamp.append(timestamp)

    fst_date = depth_obj.date[0]
    fst_time = depth_obj.time[0]
    
    date_start = f'{fst_date[6:]}{fst_date[0:2]}{fst_date[3:5]}'
    time_start = f'{fst_time[0:2]}{fst_time[3:5]}{fst_time[6:8]}'
    
    depth_obj.datetime_start = date_start + '_' + time_start
    
    depth_objs.append(depth_obj)             
            

In [41]:
from scipy import interpolate

def assign_func_from_t(time_arr, arr):
    func = interpolate.PchipInterpolator(time_arr, arr, extrapolate=False)
    return func

open_cpn_points = r'C:\YandexDisk\aa_cloudmbesproceessing\AI-69\AI69_GeophysReport\NoteToVMF_REPORT\AI69_OpenCPN_Track_Points.csv'

open_cpn_times = []
open_cpn_lons = []
open_cpn_lats = []

with open(open_cpn_points, 'r') as f3:
    f3_content = f3.read().splitlines()
    
    timestamp_prev = 0
        
    for num, line in enumerate(f3_content[1:]):
        line_content = line.split(',')
        
        datetime_obj = datetime.datetime.strptime(line_content[1], '%Y/%m/%d %H:%M:%S+00')
        datetime_obj.replace(tzinfo=datetime.timezone.utc)

        timestamp = datetime_obj.timestamp()
        

        lon = float(line_content[2])
        lat = float(line_content[3])
        
        open_cpn_times.append(timestamp)
        open_cpn_lons.append(lon)
        open_cpn_lats.append(lat)
        

lat_func = assign_func_from_t(open_cpn_times, open_cpn_lats)
lon_func = assign_func_from_t(open_cpn_times, open_cpn_lons)    

In [43]:
for depth_obj in depth_objs:
    filename = f'AI69_SES2000_{depth_obj.datetime_start}.csv'
    save_to = os.path.join(path, filename)
        
    with open(save_to, 'w') as f2:
        # f2.write('num,lat_raw,lon_raw,lat_proc,lon_proc,date,time,timestamp,sv,depth\n')
        f2.write('latWGS84,lonWGS84,date,time,timestamp,sv,depth\n')
            
        for num, _ in enumerate(depth_obj.depth):
            
            lat = lat_func(depth_obj.timestamp[num])
            lon = lon_func(depth_obj.timestamp[num])
            
            # f2.write(f'{num},{depth_obj.lat[num]:.6f},{depth_obj.lon[num]:.6f},{lat:.6f},{lon:.6f},{depth_obj.date[num]},')
            # f2.write(f'{depth_obj.time[num]},{depth_obj.timestamp[num]:.0f},{depth_obj.sv[num]},{depth_obj.depth[num]:.2f}\n')
            
            f2.write(f'{lat:.6f},{lon:.6f},{depth_obj.date[num]},')
            f2.write(f'{depth_obj.time[num]},{depth_obj.timestamp[num]:.0f},{depth_obj.sv[num]:.0f},{depth_obj.depth[num]:.2f}\n')
    